# Data setup

In [1]:
import pickle as pkl
import numpy as np
import pandas as pd

In [2]:
encoded_train, train_labels = pkl.load(open("/n/scratch2/ms994/encoded_train_cnn_lstm.pkl", "rb"))

In [3]:
encoded_valid, valid_labels = pkl.load(open("/n/scratch2/ms994/encoded_valid_cnn_lstm.pkl", "rb"))
encoded_test, test_labels = pkl.load(open("/n/scratch2/ms994/encoded_test_cnn_lstm.pkl", "rb"))

In [4]:
train_seizure_detect = test_labels[0]
#bug from earlier step, forgot to iterate on length of valid or test, used train instead, with repeat on
valid_seizure_detect =  valid_labels[0][:len(encoded_valid)]
test_seizure_detect = test_labels[0][:len(encoded_test)]

In [5]:
encoded_train.shape

(203776, 125, 32)

In [6]:
import keras

Using TensorFlow backend.


# Setup a quick network here
Train on the encoded feature vector for predicting seizure and other metrics

In [18]:
y

<tf.Tensor 'leaky_re_lu_3/LeakyRelu:0' shape=(?, 32) dtype=float32>

In [82]:
x = keras.layers.Input((125*32,))
y = keras.layers.Reshape((125,32))(x)
for i in range(2):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.CuDNNLSTM(32, return_sequences=True)(y)
    y = keras.layers.LeakyReLU()(y)
y = keras.layers.Flatten()(y)
for i in range(2):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.Dense(512)(y)
    y = keras.layers.LeakyReLU()(y)
    y = keras.layers.Dropout(0.5)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.layers.Dense(2, activation="softmax")(y)
model = keras.models.Model(inputs=x, outputs=y)
model.compile(keras.optimizers.Adam(lr=0.00005), loss="categorical_crossentropy")

In [89]:
keras.layers.SpatialDropout1D

keras.layers.core.SpatialDropout1D

In [83]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 4000)              0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 125, 32)           0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 125, 32)           128       
_________________________________________________________________
cu_dnnlstm_18 (CuDNNLSTM)    (None, 125, 32)           8448      
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 125, 32)           0         
_________________________________________________________________
batch_normalization_33 (Batc (None, 125, 32)           128       
_________________________________________________________________
cu_dnnlstm_19 (CuDNNLSTM)    (None, 125, 32)           8448      
__________

In [84]:
import imblearn
gen = imblearn.keras.BalancedBatchGenerator(encoded_train.reshape((-1,125*32)), keras.utils.to_categorical(train_labels[0], 2), )
val_gen = imblearn.keras.BalancedBatchGenerator(encoded_valid.reshape((-1,32*125)), keras.utils.to_categorical(valid_labels[0][:len(encoded_valid)], 2))

In [85]:
earlyStop = keras.callbacks.EarlyStopping(monitor="loss", patience=30, verbose=1)
modelCheck = keras.callbacks.ModelCheckpoint("/n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnly.h5", save_best_only=True, verbose=1)
history = model.fit_generator(gen, validation_data=(val_gen), epochs=1000, callbacks=[earlyStop, modelCheck])

Epoch 1/1000
962/962 [==============================] - 26s 27ms/step - loss: 0.9556 - val_loss: 0.7187

Epoch 00001: val_loss improved from inf to 0.71866, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnly.h5
Epoch 2/1000
962/962 [==============================] - 20s 21ms/step - loss: 0.8742 - val_loss: 0.7176

Epoch 00002: val_loss improved from 0.71866 to 0.71756, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnly.h5
Epoch 3/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.8337 - val_loss: 0.7064

Epoch 00003: val_loss improved from 0.71756 to 0.70640, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnly.h5
Epoch 4/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.8076 - val_loss: 0.7045

Epoch 00004: val_loss improved from 0.70640 to 0.70450, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnly.h5
Epoch 5/1000
962/962 [==============================] - 21s 21ms/ste

962/962 [==============================] - 21s 21ms/step - loss: 0.4133 - val_loss: 0.9750

Epoch 00050: val_loss did not improve from 0.70056
Epoch 51/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.4021 - val_loss: 0.9966

Epoch 00051: val_loss did not improve from 0.70056
Epoch 52/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.3959 - val_loss: 1.0044

Epoch 00052: val_loss did not improve from 0.70056
Epoch 53/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.3980 - val_loss: 1.0075

Epoch 00053: val_loss did not improve from 0.70056
Epoch 54/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.3909 - val_loss: 1.0184

Epoch 00054: val_loss did not improve from 0.70056
Epoch 55/1000
962/962 [==============================] - 20s 21ms/step - loss: 0.3839 - val_loss: 1.0398

Epoch 00055: val_loss did not improve from 0.70056
Epoch 56/1000
962/962 [==============================] - 21s 21ms/step -

962/962 [==============================] - 21s 21ms/step - loss: 0.1431 - val_loss: 1.7478

Epoch 00154: val_loss did not improve from 0.70056
Epoch 155/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.1452 - val_loss: 1.7656

Epoch 00155: val_loss did not improve from 0.70056
Epoch 156/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.1417 - val_loss: 1.7671

Epoch 00156: val_loss did not improve from 0.70056
Epoch 157/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.1399 - val_loss: 1.7939

Epoch 00157: val_loss did not improve from 0.70056
Epoch 158/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.1424 - val_loss: 1.7722

Epoch 00158: val_loss did not improve from 0.70056
Epoch 159/1000
962/962 [==============================] - 21s 21ms/step - loss: 0.1373 - val_loss: 1.7860

Epoch 00159: val_loss did not improve from 0.70056
Epoch 160/1000
962/962 [==============================] - 21s 21ms/

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["Train", "Valid"])
plt.show()

In [ ]:
model = keras.models.load_model("/n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnly.h5")

In [ ]:
test_pred = model.predict(encoded_test.reshape((-1,125*32)))

In [ ]:
len(test_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_pred.argmax(1), test_labels[0][:len(encoded_test)]))

In [60]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_pred.argmax(1), test_labels[0][:len(encoded_test)])

0.49972976051856816

In [86]:
train_pred = model.predict(encoded_train.reshape((-1, 125*32)))

In [87]:
print(classification_report(train_pred.argmax(1), train_labels[0]))

              precision    recall  f1-score   support

           0       0.59      0.98      0.73    112517
           1       0.86      0.15      0.25     91259

    accuracy                           0.61    203776
   macro avg       0.72      0.56      0.49    203776
weighted avg       0.71      0.61      0.52    203776



In [88]:
roc_auc_score(train_pred.argmax(1), train_labels[0])

0.5634694815859486

# Try some other approaches for further feature analysis from Autoencoder results

In [12]:
from sklearn.decomposition import PCA, FastICA

In [ ]:
pca = PCA()
pca.fit(encoded_train.reshape((-1, 125*32)))

In [13]:
ica = FastICA()
ica.fit(encoded_train.reshape((-1, 125*32)))

KeyboardInterrupt: 

In [15]:
pd.Series(pca.explained_variance_ratio_).sort_values(ascending=False).head()

0    0.013694
1    0.009632
2    0.009336
3    0.009239
4    0.008088
dtype: float64

In [48]:
np.broadcast_to(keras.utils.to_categorical([0,1,0,0,1],2), (5,5,2)).transpose((1,0,2))

ValueError: all elements of broadcast shape must be non-negative

# try a sequence only network?

In [7]:
seizure_time_labels = np.broadcast_to(keras.utils.to_categorical(train_labels[0], 2), (125, len(train_labels[0]), 2), ).transpose((1,0,2))

In [8]:
seizure_time_labels.shape

(203776, 125, 2)

In [9]:
valid_seizure_time_labels = keras.utils.to_categorical(valid_labels[0][:len(encoded_valid)], 2)
valid_seizure_time_labels = np.broadcast_to(valid_seizure_time_labels, (125, len(encoded_valid), 2)).transpose((1,0,2))

In [10]:
import imblearn
gen = imblearn.keras.BalancedBatchGenerator(encoded_train.reshape((-1,125*32)), seizure_time_labels, )
val_gen = imblearn.keras.BalancedBatchGenerator(encoded_valid.reshape((-1,32*125)), valid_seizure_time_labels)

ValueError: Unknown label type: 'unknown'

In [11]:
x = keras.layers.Input((125*32,))
y = keras.layers.Reshape((125,32))(x)
for i in range(2):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.CuDNNLSTM(32, return_sequences=True)(y)
    y = keras.layers.LeakyReLU()(y)
for i in range(2):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.Dense(32)(y)
    y = keras.layers.LeakyReLU()(y)
    y = keras.layers.Dropout(0.5)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.layers.Dense(2, activation="softmax")(y)
model = keras.models.Model(inputs=x, outputs=y)
model.compile(keras.optimizers.Adam(lr=0.00005), loss="categorical_crossentropy")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4000)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 125, 32)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 32)           128       
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 125, 32)           8448      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 125, 32)           0         
_________________________________________________________________
batch_normalization_2

In [12]:
encoded_train.reshape((-1,125*32)).shape

(203776, 4000)

In [13]:
earlyStop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=30, verbose=1)
modelCheck = keras.callbacks.ModelCheckpoint("/n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnlyOverTime.h5", save_best_only=True, verbose=1)
history = model.fit(encoded_train.reshape((-1,125*32)), seizure_time_labels, batch_size=32, validation_data=(encoded_valid.reshape((-1,125*32)), valid_seizure_time_labels), epochs=1000, callbacks=[earlyStop, modelCheck])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 203776 samples, validate on 81152 samples
Epoch 1/1000
203776/203776 [==============================] - 133s 655us/step - loss: 0.4285 - val_loss: 0.2251

Epoch 00001: val_loss improved from inf to 0.22514, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnlyOverTime.h5
Epoch 2/1000
203776/203776 [==============================] - 126s 619us/step - loss: 0.2760 - val_loss: 0.2232

Epoch 00002: val_loss improved from 0.22514 to 0.22325, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnlyOverTime.h5
Epoch 3/1000
203776/203776 [==============================] - 126s 620us/step - loss: 0.2730 - val_loss: 0.2230

Epoch 00003: val_loss improved from 0.22325 to 0.22300, saving model to /n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnlyOverTime.h5
Epoch 4/1000
203776/203776 [==============================] - 126s 620us/step

203776/203776 [==============================] - 127s 621us/step - loss: 0.2677 - val_loss: 0.2212

Epoch 00040: val_loss did not improve from 0.22014
Epoch 41/1000
203776/203776 [==============================] - 127s 621us/step - loss: 0.2676 - val_loss: 0.2216

Epoch 00041: val_loss did not improve from 0.22014
Epoch 42/1000
203776/203776 [==============================] - 126s 620us/step - loss: 0.2676 - val_loss: 0.2211

Epoch 00042: val_loss did not improve from 0.22014
Epoch 43/1000
203776/203776 [==============================] - 126s 620us/step - loss: 0.2677 - val_loss: 0.2216

Epoch 00043: val_loss did not improve from 0.22014
Epoch 44/1000
203776/203776 [==============================] - 127s 621us/step - loss: 0.2676 - val_loss: 0.2206

Epoch 00044: val_loss did not improve from 0.22014
Epoch 45/1000
203776/203776 [==============================] - 126s 621us/step - loss: 0.2676 - val_loss: 0.2205

Epoch 00045: val_loss did not improve from 0.22014
Epoch 46/1000
203776/203

203776/203776 [==============================] - 126s 619us/step - loss: 0.2643 - val_loss: 0.2250

Epoch 00090: val_loss did not improve from 0.22014
Epoch 91/1000
203776/203776 [==============================] - 126s 619us/step - loss: 0.2642 - val_loss: 0.2249

Epoch 00091: val_loss did not improve from 0.22014
Epoch 92/1000
203776/203776 [==============================] - 126s 619us/step - loss: 0.2640 - val_loss: 0.2284

Epoch 00092: val_loss did not improve from 0.22014
Epoch 93/1000
203776/203776 [==============================] - 126s 618us/step - loss: 0.2639 - val_loss: 0.2291

Epoch 00093: val_loss did not improve from 0.22014
Epoch 94/1000
203776/203776 [==============================] - 126s 619us/step - loss: 0.2637 - val_loss: 0.2281

Epoch 00094: val_loss did not improve from 0.22014
Epoch 95/1000
203776/203776 [==============================] - 126s 619us/step - loss: 0.2637 - val_loss: 0.2266

Epoch 00095: val_loss did not improve from 0.22014
Epoch 96/1000
203776/203

KeyboardInterrupt: 

In [14]:
model = keras.models.load_model("/n/scratch2/ms994/autoEncoderOutputCNNLSTMSeizureOnlyOverTime.h5")

In [16]:
pred = model.predict(encoded_test.reshape((-1,125*32)))

In [17]:
pred.shape

(81152, 125, 2)

In [19]:
from sklearn.metrics import classification_report, roc_auc_score

In [22]:
print(classification_report(test_labels[0][:len(encoded_test)], pred.argmax(1)))

ValueError: Classification metrics can't handle a mix of binary and multiclass-multioutput targets